In [ ]:
import seaborn as sns 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../data/NEW2.csv')
df = df[['price', 'grade', 'bedrooms', 'sqft_living', 'sqft_lot15', 'sqft_basement', 'zipcode']]
print(df.shape)
def get_clean_data(df):
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        mean = df[col].mean()
        std = df[col].std()
        df = df[(df[col] < (mean + 5*std)) & (df[col] > (mean - 5*std))]
    return df

df = get_clean_data(df)
print(df.shape)

In [ ]:
def get_heatmap(df):
    '''This function will create a heatmap to compare the different independent variables of our model'''
    sns.heatmap(df.drop(['price', 'zipcode'], axis = 1).corr(), annot = True)
    plt.title('Correlations of the Independent Variables')
    
get_heatmap(df)

### Regression

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import statsmodels.api as sm 
import pandas as pd

In [ ]:
X = df[[i for i in df.columns if i != 'price']]
y = df[['price']]/1000000
X = sm.add_constant(X)
x_train, x_test, y_train, y_test = train_test_split(X,y, random_state = 0, train_size = .8, stratify = X.zipcode.values)
x_train.drop('zipcode', axis = 1, inplace =True)
x_test.drop('zipcode', axis = 1, inplace = True)

def regression(x_train= x_train,y_train=y_train):
    '''This function will create and retrun a regression model'''
    linear = sm.OLS(y_train,x_train, hasconst = True).fit() 
    summary = linear.summary()
    print(summary)
    
    return linear 

model = regression()



In [ ]:
def scatter(model, x_test=x_test, y_test=y_test): 
    '''This function will create a scatter plot for the residuals of the model'''
    predictions = model.predict(x_test)
    residuals = predictions - y_test.values.squeeze() 

    plt.scatter(predictions, residuals)
    plt.hlines(y = 0, xmin = predictions.min(), xmax = predictions.max())
    plt.vlines(x = 1,ymin = residuals.min(), ymax = residuals.max())
    plt.xlabel('Predictions ($ millions)'); plt.ylabel('Residuals (predicted - actual)'); 
    loss = mean_squared_error(y_test, predictions)*1000000
    plt.title(f'Residuals for Linear Regression Model | RMSE: {int(round(loss, 0))}')
    
scatter(model = model)